# Bearbeitungshinweise
Im Notebook werden verschiedene Fragestellungen an einem Datensatz untersucht. Die Aufgaben, die sie bearbeiten sollen, ergänzen diese Untersuchungen. Unter jeder Aufgabe befindet sich eine (Code-)Zelle, die etwa so aussieht:

In [1]:
# <IHRE LÖSUNG HIER>

Dort tragen Sie bitte ihren Python Code ein. Bei manchen Aufgaben wird auch kein Code verlangt, sondern eine (kurze) Erklärung. In diesem Fall konvertieren Sie die Zelle bitte zu einer Markdown-Zelle ("Esc" und dann "M" drücken - "Esc" und dann "Y" konvertiert wieder zu einer Code-Zelle) und tragen dort Ihre Antwort ein.

Falls Sie keine lokale Python Installation haben, können Sie das Notebook auch in Google Colab öffnen und bearbeiten (Google Account benötigt):

<a target="_blank" href="https://colab.research.google.com/github/djanka2/statistik-uebungen/blob/main/02%20-%20Korrelation%20und%20Regression.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

# Übung 2: Korrelation und Regression

## World Happiness Report

Der World Happiness Report (https://worldhappiness.report/) ist ein jährlich vom Sustainable Development Solutions Network der Vereinten Nationen veröffentlichter Bericht. Der Bericht enthält Ranglisten zur Lebenszufriedenheit in verschiedenen Ländern der Welt und Datenanalysen aus verschiedenen Perspektiven. 

Die wichtigste Frage der Umfrage, auf der der Bericht basiert, ist die so genannte *Cantril-Leiter*: Die Befragten werden gebeten, sich eine Leiter vorzustellen, bei der das bestmögliche Leben für sie eine 10 und das schlechtestmögliche Leben eine 0 ist. Anschließend werden sie gebeten, ihr eigenes aktuelles Leben auf dieser Skala von 0 bis 10 zu bewerten.

## Die Daten

Wir laden den Bericht aus dem Jahr 2022.

In [2]:
import pandas as pd
import plotly.express as px

whr = pd.read_csv("https://raw.githubusercontent.com/djanka2/statistik-uebungen/main/Daten/WHR2022.csv")

### ✏️ Aufgabe 1
Welche Dimension haben die Daten (d.h. wie viele statistische Einheiten und Variablen werden betrachtet)? Was repräsentiert eine Zeile in den Daten? 
Was bedeuten die Spalten? Details können Sie in der Datei WHR+22.pdf auf S.15 und S.21 nachschlagen!

In [3]:
# <IHRE LÖSUNG HIER>
print(whr.info())
print("\n--------------------\n")
print(whr.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089 entries, 0 to 2088
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country name                       2089 non-null   object 
 1   year                               2089 non-null   int64  
 2   Life Ladder                        2089 non-null   float64
 3   Log GDP per capita                 2062 non-null   float64
 4   Social support                     2076 non-null   float64
 5   Healthy life expectancy at birth   2031 non-null   float64
 6   Freedom to make life choices       2057 non-null   float64
 7   Generosity                         2009 non-null   float64
 8   Perceptions of corruption          1976 non-null   float64
 9   Positive affect                    2065 non-null   float64
 10  Negative affect                    2073 non-null   float64
 11  Confidence in national government  1873 non-null   float

### ✏️ Aufgabe 2
Der Datensatz erstreckt sich über viele Jahre, aber wir sind nur an den Daten aus dem Jahr 2021 interessiert. Filtern Sie den DataFrame `whr` für das Jahr 2021, wählen Sie alle Spalten aus und weisen Sie das Ergebnis einem DataFrame mit dem Namen `whr_2021` zu.

In [4]:
# <IHRE LÖSUNG HIER>
whr_2021 = whr[whr['year'] == 2021]

### ✏️ Aufgabe 3
Wie hoch ist die durchschnittliche Lebenserwartung `Healthy life expectancy at birth` über alle Länder? Können Sie aus den Daten auch die durchschnittliche Lebenserwartung aller Menschen berechnen? (wenn ja: wie? wenn nein: warum nicht?)

In [5]:
# <IHRE LÖSUNG HIER>
print(whr['Healthy life expectancy at birth'].mean(axis=None))
"""
Es macht keinen Sinn die durchschnittliche Lebenserwartung für alle Menschen zu berechnen, indem man die Lebenserwartung jedes Landes nimmt 
und sie einfach zusammenmittelt. Dies liegt daran, dass sich die Bevölkerungen der verschiedenen Länder stark unterscheiden können, 
und einige Länder möglicherweise eine viel größere Bevölkerung haben als andere. Um die durchschnittliche Lebenserwartung für alle Menschen zu berechnen,
müsste ein gewichteter Durchschnitt verwendet werden, der die Bevölkerungsgröße jedes Landes berücksichtigt. Diese ist in den vorliegenden Daten jedoch nicht
vorhanden. Darüber hinaus können auch andere Faktoren wie sozioökonomischer Status, Zugang zur Gesundheitsversorgung und Lebensstilwahl die Lebenserwartung beeinflussen, 
sodass eine komplexere Analyse erforderlich wäre, um die durchschnittliche Lebenserwartung für alle Menschen genau schätzen zu können.
"""

63.1803259342393


'\nEs macht keinen Sinn die durchschnittliche Lebenserwartung für alle Menschen zu berechnen, indem man die Lebenserwartung jedes Landes nimmt \nund sie einfach zusammenmittelt. Dies liegt daran, dass sich die Bevölkerungen der verschiedenen Länder stark unterscheiden können, \nund einige Länder möglicherweise eine viel größere Bevölkerung haben als andere. Um die durchschnittliche Lebenserwartung für alle Menschen zu berechnen,\nmüsste ein gewichteter Durchschnitt verwendet werden, der die Bevölkerungsgröße jedes Landes berücksichtigt. Diese ist in den vorliegenden Daten jedoch nicht\nvorhanden. Darüber hinaus können auch andere Faktoren wie sozioökonomischer Status, Zugang zur Gesundheitsversorgung und Lebensstilwahl die Lebenserwartung beeinflussen, \nsodass eine komplexere Analyse erforderlich wäre, um die durchschnittliche Lebenserwartung für alle Menschen genau schätzen zu können.\n'

WIr untersuchen nun für das Jahr 2021 folgende Frage:
>Gibt es einen Zusammenhang zwischen der Lebenserwartung ("Healthy life expectancy at birth") und der Lebenszufriedenheit ("Life Ladder") in einem Land? 

Um das herauszufinden, visualisieren wir die Daten. So wie man die Verteilung *einer* Variablen mit einem Histogramm oder einem Dichteplot visualisiert, haben Statistiker eine allgemeine Methode für die Visualisierung der Beziehung zwischen zwei numerischen Variablen entwickelt: das Streudiagramm. Das Streudiagramm (oder Scatterplot) wurde auch als die ["nützlichste Erfindung in der Geschichte der statistischen Grafik"](https://onlinelibrary.wiley.com/doi/abs/10.1002/jhbs.20078) bezeichnet. Es handelt sich um ein einfaches zweidimensionales Diagramm, bei dem die beiden Koordinaten jedes Punktes die Werte von zwei Variablen darstellen, die bei einer einzigen Beobachtung gemessen wurden.

Wir erzeugen wir einen Streuplot mit plotly mit folgendem Befehl:

In [19]:
px.scatter(data_frame=whr, x="Healthy life expectancy at birth", y="Life Ladder")


Wenn Sie die Maus über das Diagramm bewegen, werden zu jedem Punkt die x- und y-Werte angezeigt - praktisch. Noch besser wäre es, wenn wir zusätzlich zu jedem Punkt sehen würden, welches Land sich dahinter verbirgt. Dies können wir erreichen, indem wir dem Befehl `px.scatter` über das Argument `hover_name=` den Spaltennamen des DataFrames übergeben, der zu jedem Punkt den Namen des Landes enthält, hier also: `Country name`:

In [7]:
px.scatter(data_frame=whr, x="Healthy life expectancy at birth", y="Life Ladder", hover_name="Country name")

### ✏️ Aufgabe 4
In welchem Land ist die Lebenserwartung am geringsten, in welchem am höchsten? In welchem Land sind die Menschen am zufriedensten mit ihrem Leben, in welchem am unzufriedensten?

In [8]:
# <IHRE LÖSUNG HIER>
print(whr.loc[whr['Healthy life expectancy at birth'] == whr['Healthy life expectancy at birth'].max(axis=None), ['Country name']].to_string(index=False))
print(whr.loc[whr['Healthy life expectancy at birth'] == whr['Healthy life expectancy at birth'].min(axis=None), ['Country name']].to_string(index=False))

print(whr.loc[whr['Life Ladder'] == whr['Life Ladder'].max(axis=None), ['Country name']].to_string(index=False))
print(whr.loc[whr['Life Ladder'] == whr['Life Ladder'].min(axis=None), ['Country name']].to_string(index=False))


Country name
       Japan
Country name
       Haiti
Country name
     Denmark
Country name
     Lebanon


Die Punkte im Streudiagramm liegen zwar verteilt in der Ebene, aber man kann doch erkennen, dass im Allgemeinen mit zunehmender Lebenserwartung die Lebenszufriedenheit zunimmt. Dies quantifizieren wir nun mit Hilfe des **Korrelationskoeffizienten**.

Es gibt unterschiedliche Möglichkeiten, diesen in Python zu berechnen. Wir nutzen die `.corr` Funktion von Pandas:

In [9]:
whr["Healthy life expectancy at birth"].corr(whr["Life Ladder"])

0.7137951622549499

Der Wert besagt, dass es eine positive Korrelation zwischen Lebenserwartung und Lebenszufriedenheit gibt, dass also mit steigender Lebenserwartung auch die Lebenszufriedenheit steigt - und umgekehrt. Dieser Zusammenhang gilt nicht in jedem Einzelfall wie man leicht im Streudiagramms sieht. Eine gute Möglichkeit, sich die Stärke einer Korrelation zu veranschaulichen ist, die Regressionsgerade zu plotten. Würden alle Punkte genau auf der Geraden liegen, so hätte man eine perfekte Korrelation: +1, wenn die Gerade eine positive Steigung hat oder -1, wenn die Gerade eine negative Steigung hat (dies kommt bei realistischen Daten quasi nie vor).

plotly erlaubt es, die Regressionsgerade direkt im Plot zu visualisieren. Dafür muss das Modul `statsmodels` installiert sein (in der Anaconda Konsole `pip install statsmodels` eingeben). Die einfache Regressionsgerade wird über das Argument `trendline="ols"` aktiviert (ols steht für ordinary least squares).

In [21]:
px.scatter(data_frame=whr, x="Healthy life expectancy at birth", y="Life Ladder", hover_name="Country name", trendline="ols")

Die positive Steigung der Geraden impliziert, dass eine *positive* Korrelation vorliegt. Die *Stärke* der Korrelation ist dadurch bestimmt, wie nahe an der Geraden die Punkte typischerweise liegen. Regressionsgeraden sind also dazu geeignet "Korrelationen sichtbar zu machen". 

Praktisch: Wenn Sie die Maus über die Gerade bewegen, wird die Geradengleichung, sowie das R² Gütmaß angezeigt. Zur Erinnerung: R² ist ein Wert zwischen 0 und 1, der aussagt, wie viel der Varianz in den Daten durch den Prediktor (hier: Lebenserwartung) erklärt wird. R² ist der quadrierte Korrelationskoeffizient.  

Wenn wir die Koeffizienten der Geraden nun automatisch auslesen möchten, können wir dies wie folgt tun. Wir speichern den *Plot* in einer Variable:

In [ ]:
p = px.scatter(data_frame=whr, x="Healthy life expectancy at birth", y="Life Ladder", hover_name="Country name", trendline="ols")

Wenn Sie diesen Code ausführen, werden Sie merken, dass der Plot nicht angezeigt wird (er wird erzeugt und in der Variable `p` gespeichert). Wir können die Koeffizienten $\alpha$ uund $\beta$ nun mittels folgenden Befehlen aus `p` extrahieren:

In [ ]:
result = px.get_trendline_results(p)
result.iloc[0,0].params

array([-1.83888536,  0.11569767])

Übrigens: mit `p.show()` könnten Sie den Plot jederzeit anzeigen.

### ✏️ Aufgabe 5
1. Berechnen Sie die Koeffizienten $\alpha$ und $\beta$ der Regressionsgeraden $\hat{y}=\alpha+\beta x$ mittels der Formeln aus der Vorlesung und vergleichen Sie mit den Werten oben:
    - $\beta = r_{xy}\frac{\tilde{s}_y}{\tilde{s}_x}$
    - $\alpha = \bar{y} - \beta \bar{x}$

2. Welchen Wert für Lebenszufriedenheit sagt das Modell für eine Lebenserwartung von 80 Jahren voraus?

In [ ]:
# <IHRE LÖSUNG HIER>
beta = whr['Healthy life expectancy at birth'].corr(whr['Life Ladder']) * (whr['Life Ladder'].std() / whr['Healthy life expectancy at birth'].std())
alpha = whr['Life Ladder'].mean() - beta * whr['Healthy life expectancy at birth'].mean()

life_ladder_life_expectancy_eighty = alpha + beta * 80
print(life_ladder_life_expectancy_eighty)

7.401240096089374


---
Der Korrelation zwischen Lebenserwartung und Zufriedenheit war ein Beispiel für eine positive Korrelation. Natürlich können Sie für jedes beliebige Paar von Variablen die Korrelation berechnen.

### ✏️ Aufgabe 6
Zwischen welchen Variablen im Datensatz `whr_2021` vermuten Sie eine *negative* Korrelation? Warum? Verifizieren Sie ihre Vermutung, indem Sie
1. den Korrelationskoeffizienten berechnen und
2. die beiden betreffenden Merkmale sowie die Regressionsgerade in einem Streuplot anzeigen.

In [ ]:
# <IHRE LÖSUNG HIER>
"""
Hohe Wahrnehmung von Korruption in einem Land ist mit niedrigeren Ebenen des subjektiven Wohlbefindens verbunden. 
Korruption untergräbt das Vertrauen in Institutionen, schafft wirtschaftliche Instabilität und führt zu einer unfairen Verteilung von Ressourcen. 
Im Gegensatz dazu haben Länder mit niedriger wahrgenommener Korruption tendenziell höhere Ebenen des Wohlbefindens, 
da Bürger optimistischer in die Zukunft blicken. Die Reduzierung von Korruption kann dazu beitragen, das allgemeine Wohlbefinden und Glück in der Gesellschaft zu verbessern.
"""
whr["Life Ladder"].corr(whr["Perceptions of corruption"])
px.scatter(data_frame=whr, x="Life Ladder", y="Perceptions of corruption", hover_name="Country name", trendline="ols")